## SI 650 / EECS 549: Homework 3 Part 2

Homework 3 Part 2 will have you working with deep learning models in a variety of ways. You will likely need to run this on Great Lakes unless you have access to a GPU elsewhere (or be prepared to wait a long time). You should have completed Parts 1 and 2 before attempting this notebook to familiarize yourself with how PyTerrier works.

In Part 3, you'll try the following tasks:
 - Use a large language model to re-rank content
 - Use a text-to-text model to perform query augmentation
 - Train a deep learning IR model and compare its performance.
 
The first two of these tasks will rely on models that we've pretrained for you. However, we've also provided code for how to train these. In the third task, you'll do one simple training in evaluate.

For the first two tasks, we've provided most of the code. *You are expected to submit results showing that you have successfully executed it*. You'll need to understand some of the code to complete task 3, which requires you to write new code.

As with the past notebooks, you'll need to have `JAVA_HOME` set, which will need to be run on Great Lakes. You can potentially set it in the notebook with a Jupyter command like
```
!export JAVA_HOME=/fill/in/the/path/to/the/JDK/here
```
by first figuring out where the JDK is installed. (This is setting the `JAVA_HOME` environment variable in the unix way). 

You can work on Parts 1 and 2 separately, so feel free to do the GPU-part (Part 2) when the resources are available on Great Lakes and the non-GPU part on your laptop or on a non-GPU Great Lakes machine (which likely will be much faster). Note that there will be no extensions due to Great Lakes bottlenecks, so you are encouraged to complete Part 2 as soon as possible.

### Install the PyTerrier extensions

You'll need two extensions for [OpenNIR](https://opennir.net/) and [doc2query](https://github.com/terrierteam/pyterrier_doc2query). We've provided the package install commands in comments below.

In [1]:
!module load openjdk
# !load openjdk/11.0.2

In [2]:
!pip install --upgrade git+https://github.com/Georgetown-IR-Lab/OpenNIR
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_doc2query.git

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/Georgetown-IR-Lab/OpenNIR to /tmp/pip-req-build-glj3sdse
  Running command git clone -q https://github.com/Georgetown-IR-Lab/OpenNIR /tmp/pip-req-build-glj3sdse
  Resolved https://github.com/Georgetown-IR-Lab/OpenNIR to commit 88a4679372f471a04d284a99404ffce2b7a1dc49
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/terrierteam/pyterrier_doc2query.git to /tmp/pip-req-build-42dwmdjl
  Running command git clone -q https://github.com/terrierteam/pyterrier_doc2query.git /tmp/pip-req-build-42dwmdjl
  Resolved https://github.com/terrierteam/pyterrier_doc2query.git to commit 247242e04a8fa6476ad1487c393bda5db226705c


In [3]:
!pip install python-terrier

Defaulting to user installation because normal site-packages is not writeable


## Getting started

Start PyTerrier as we have in past notebooks.

In [4]:
# !export JAVA_HOME=/fill/in/the/path/to/the/JDK/here
!echo $JAVA_HOME

/sw/pkgs/arc/openjdk/jdk-18.0.1.1


In [5]:
# !export JAVA_HOME=/usr/lib/jvm/java-17-oracle/
import os
# os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-17-oracle/'
os.environ['JAVA_HOME'] = '/sw/pkgs/arc/openjdk/jdk-18.0.1.1'
os.environ['CUDA_VISIBLE_DEVICES'] = '6,7'


In [6]:
import pyterrier as pt
if not pt.started():
    pt.init(tqdm="notebook")
import onir_pt
import pyterrier_doc2query
import os
import pandas as pd

PyTerrier 0.8.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


### [TREC-COVID19](https://ir.nist.gov/covidSubmit/) Dataset download

The following cell downloads the [TREC-COVID19](https://ir.nist.gov/covidSubmit/) dataset that we will use periodically throughout this notebook.

In [7]:
dataset = pt.datasets.get_dataset('irds:cord19/trec-covid')
topics = dataset.get_topics(variant='description')
qrels = dataset.get_qrels()

/home/yianchu/.local/lib/python3.9/site-packages/pyterrier/datasets.py:435: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(df.columns.difference(['qid','query']), 1, inplace=True)


# Task 1: Build the inverted index for the TREC-COVID19 collection (5 points)

Build the index for the TREC Covid-19 (CORD19) data like we have in past notebooks but without any fancy options (e.g., no positional indexing).

In [8]:
cord19 = pt.datasets.get_dataset('irds:cord19/trec-covid')

pt_index_path = './terrier_cord19'

if not os.path.exists(pt_index_path + "/data.properties"):
    # create the index, using the IterDictIndexer indexer 
    indexer = pt.index.IterDictIndexer(pt_index_path, blocks=True)

    # we give the dataset get_corpus_iter() directly to the indexer
    # while specifying the fields to index and the metadata to record
    index_ref = indexer.index(cord19.get_corpus_iter(), 
                              fields=('abstract',), 
                              meta=('docno',))

else:
    # if you already have the index, use it.
    index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")

index = pt.IndexFactory.of(index_ref)
# TODO: build the CORD19 index, and have it referenced as "index"
# for the code in later cells to use

cord19/trec-covid documents:   0%|          | 0/192509 s<?, ?it/s]

/tmp/ipykernel_1460897/2564545616.py:11: DeprecationWarning: specifying meta and meta_lengths in IterDictIndexer.index() is deprecated, use constructor instead
  index_ref = indexer.index(cord19.get_corpus_iter(),


02:03:25.785 [ForkJoinPool-1-worker-1] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (6iu1dtyl) - further warnings are suppressed
02:04:19.589 [ForkJoinPool-1-worker-1] WARN org.terrier.structures.indexing.Indexer - Indexed 54937 empty documents
02:04:20.469 [ForkJoinPool-1-worker-1] ERROR org.terrier.structures.indexing.Indexer - Could not finish MetaIndexBuilder: 
java.io.IOException: Key 8lqzfj2e is not unique: 37597,11755
For MetaIndex, to suppress, set metaindex.compressed.reverse.allow.duplicates=true
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.mergeTwo(FSOrderedMapFile.java:1374)
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.close(FSOrderedMapFile.java:1308)
	at org.terrier.structures.indexing.BaseMetaIndexBuilder.close(BaseMetaIndexBuilder.java:321)
	at org.terrier.structures.indexing.classical.BlockIndexer.createDirectIndex(BlockIndexer.java:472)
	at org.terrier.structures.indexing

## Using an untuned Re-rankers

This notebook will have you work with few neural re-ranking methods that you've used in class. We can build them from scratch using `onir_pt.reranker` or load them from pretrained models. The models we load from scratch won't have been trained to do IR (yet), however.

And OpenNIR reranking model consists of:
 - `ranker` (e.g., `drmm`, `knrm`, or `pacrr`). This defines the neural ranking architecture. We discussed the `knrm` approach in class.
 - `vocab` (e.g., `wordvec_hash`, or `bert`). This defines how text is encoded by the model. This approach makes it easy to swap out different text representations. 
 
Let's start with the `knrm` method we discussed in class:

In [9]:
knrm = onir_pt.reranker('knrm', 'wordvec_hash', text_field='abstract')

config file not found: config
[2022-11-01 02:04:28,622][WordvecHashVocab][DEBUG] [starting] reading cached at /home/yianchu/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p
[2022-11-01 02:04:29,983][WordvecHashVocab][DEBUG] [finished] reading cached at /home/yianchu/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p [1.37s]


Let's look at how well this model work at ranking compared with our default `BatchRetrieve`

In [10]:
br = pt.BatchRetrieve(index) % 100
pipeline = br >> pt.text.get_text(dataset, 'abstract') >> knrm
pt.Experiment(
    [br, pipeline],
    topics,
    qrels,
    names=['DPH', 'DPH >> KNRM'],
    eval_metrics=["map", "ndcg", 'ndcg_cut.10', 'P.10', 'mrt']
)

[2022-11-01 02:04:40,430][onir_pt][ERROR] gpu=True, but CUDA is not available. Falling back on CPU.
[2022-11-01 02:04:40,432][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/1250 s<?, ?it/s]

[2022-11-01 02:04:43,692][onir_pt][DEBUG] [finished] batches: [3.26s] [1250it] [383.53it/s]


,name,map,ndcg,ndcg_cut.10,P.10,mrt
0,DPH,0.068056,0.165653,0.609058,0.658,61.876211
1,DPH >> KNRM,0.054809,0.145486,0.359992,0.450,118.679645


The `knrm` models' performance is lower! The mode doesn't work very well because it hasn't yet been trained for IR; it's using random weights to combine the scores from the similarity matrix--but this is at least a start.

## Loading a trained re-ranker

You can train re-ranking models in PyTerrier using the `fit` method. Here's an example of how to train the `knrm` model on the MS MARCO dataset, which is a large IR collection.

```python
# transfer training signals from a medical sample of MS MARCO
from sklearn.model_selection import train_test_split
train_ds = pt.datasets.get_dataset('irds:msmarco-passage/train/medical')
train_topics, valid_topics = train_test_split(train_ds.get_topics(), test_size=50, random_state=42) # split into training and validation sets

# Index MS MARCO
indexer = pt.index.IterDictIndexer('./terrier_msmarco-passage')
tr_index_ref = indexer.index(train_ds.get_corpus_iter(), fields=('text',), meta=('docno',))

pipeline = (pt.BatchRetrieve(tr_index_ref) % 100 # get top 100 results
            >> pt.text.get_text(train_ds, 'text') # fetch the document text
            >> pt.apply.generic(lambda df: df.rename(columns={'text': 'abstract'})) # rename columns
            >> knrm) # apply neural re-ranker

pipeline.fit(
    train_topics,
    train_ds.get_qrels(),
    valid_topics,
    train_ds.get_qrels())
```

Training deep learning models takes a bit of time (especially for large datasets like MS MARCO), so we've provided a model that's already been trained for you to download.

In [11]:
del knrm # free up the memory before loading a new version of the ranker (helpful for the GPU)
knrm = onir_pt.reranker.from_checkpoint('https://macavaney.us/knrm.medmarco.tar.gz', text_field='abstract', 
                                        expected_md5="d70b1d4f899690dae51161537e69ed5a")

[2022-11-01 02:04:43,828][onir_pt][INFO] using cached checkpoint: /home/yianchu/data/onir/model_checkpoints/b7694d2fb4d4f8218e11734de239ad30
[2022-11-01 02:04:43,833][WordvecHashVocab][DEBUG] [starting] reading cached at /home/yianchu/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p
[2022-11-01 02:04:45,184][WordvecHashVocab][DEBUG] [finished] reading cached at /home/yianchu/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p [1.35s]


In [12]:
pipeline2 = br >> pt.text.get_text(dataset, 'abstract') >> knrm
pt.Experiment(
    [br, pipeline2],
    topics,
    qrels,
    names=['DPH', 'DPH >> KNRM'],
    baseline=0,
    eval_metrics=["map", "ndcg", 'ndcg_cut.10', 'P.10', 'mrt']
)

[2022-11-01 02:04:54,954][onir_pt][ERROR] gpu=True, but CUDA is not available. Falling back on CPU.
[2022-11-01 02:04:54,956][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/1250 s<?, ?it/s]

[2022-11-01 02:04:58,728][onir_pt][DEBUG] [finished] batches: [3.77s] [1250it] [331.49it/s]


,name,map,P.10,ndcg,ndcg_cut.10,mrt,map +,map -,map p-value,P.10 +,P.10 -,P.10 p-value,ndcg +,ndcg -,ndcg p-value,ndcg_cut.10 +,ndcg_cut.10 -,ndcg_cut.10 p-value
0,DPH,0.068056,0.658,0.165653,0.609058,52.209043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DPH >> KNRM,0.065103,0.598,0.160561,0.532655,126.053247,20.0,30.0,0.095852,12.0,26.0,0.024604,20.0,30.0,0.028273,20.0,30.0,0.005972


The tuned performance is a little better than before, but `knrm` still underperforms our first-stage ranking model.

## Reranking with BERT

Large language models such as [BERT](https://arxiv.org/abs/1810.04805) are much more powerful neural models that have been shown to be effective for ranking like we discussed in class. 

Like with `knrm`, we'll start by using BERT for re-ranking with its initial parameters. These parameters have been turned for the masked language modeling (i.e., filling a word in the blank) and predicting the next sentence--but have not been tuned for IR at all.

In [13]:
del knrm # clear out memory from KNRM (useful for GPU)
vbert = onir_pt.reranker('vanilla_transformer', 'bert', text_field='abstract', vocab_config={'train': True})

Let's see how this non-IR trained model does on CORD10 data

In [14]:
pipeline3 = br % 100 >> pt.text.get_text(dataset, 'abstract') >> vbert
pt.Experiment(
    [br, pipeline3],
    topics,
    qrels,
    names=['DPH', 'DPH >> VBERT'],
    baseline=0,
    eval_metrics=["map", "ndcg", 'ndcg_cut.10', 'P.10', 'mrt']
)

[2022-11-01 02:05:09,095][onir_pt][ERROR] gpu=True, but CUDA is not available. Falling back on CPU.
[2022-11-01 02:05:09,100][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/1250 s<?, ?it/s]

[2022-11-01 02:33:43,707][onir_pt][DEBUG] [finished] batches: [28:35] [1250it] [ 1.37s/it]


,name,map,P.10,ndcg,ndcg_cut.10,mrt,map +,map -,map p-value,P.10 +,P.10 -,P.10 p-value,ndcg +,ndcg -,ndcg p-value,ndcg_cut.10 +,ndcg_cut.10 -,ndcg_cut.10 p-value
0,DPH,0.068056,0.658,0.165653,0.609058,46.691339,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DPH >> VBERT,0.056413,0.458,0.147048,0.374197,34342.213411,8.0,42.0,0.000006,6.0,37.0,9.462198e-07,5.0,45.0,0.000001,8.0,41.0,4.134415e-09


As we see, although the ERT model is pre-trained for recognizing language, it doesn't do very well at ranking on our benchmark. To get better performance, we'll need to tune for the task of relevance ranking.

We can train the model for ranking (as shown above for KNRM) or we can download a trained model. Here, we will use the [SLEDGE](https://arxiv.org/abs/2010.05987) model, which is a BERT model trained on scientific text and tuned on medical queries.

In [15]:
vbert = onir_pt.reranker.from_checkpoint('https://macavaney.us/scibert-medmarco.tar.gz', 
                                         text_field='abstract', expected_md5="854966d0b61543ffffa44cea627ab63b")

[2022-11-01 02:33:43,877][onir_pt][INFO] using cached checkpoint: /home/yianchu/data/onir/model_checkpoints/1153203ea2abbdf7f760fa6956a83d83


Let's run another experiment to see how this new model trained for IR does.

In [16]:
pipeline4 = br % 100 >> pt.text.get_text(dataset, 'abstract') >> vbert
pt.Experiment(
    [br, pipeline4],
    topics,
    qrels,
    names=['DPH', 'DPH >> Trained-BERT'],
    baseline=0,
    eval_metrics=["map", "ndcg", 'ndcg_cut.10', 'P.10', 'mrt']
)

[2022-11-01 02:34:10,491][onir_pt][ERROR] gpu=True, but CUDA is not available. Falling back on CPU.
[2022-11-01 02:34:10,495][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/1250 s<?, ?it/s]

[2022-11-01 03:00:17,566][onir_pt][DEBUG] [finished] batches: [26:07] [1250it] [ 1.25s/it]


,name,map,P.10,ndcg,ndcg_cut.10,mrt,map +,map -,map p-value,P.10 +,P.10 -,P.10 p-value,ndcg +,ndcg -,ndcg p-value,ndcg_cut.10 +,ndcg_cut.10 -,ndcg_cut.10 p-value
0,DPH,0.068056,0.658,0.165653,0.609058,50.628312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DPH >> Trained-BERT,0.075710,0.770,0.173079,0.701995,31391.654690,36.0,14.0,0.001278,28.0,11.0,0.000851,36.0,14.0,0.010118,31.0,19.0,0.012156


Training helped a lot! We're able to improve upon the initial ranking from `BatchRetrieve`. However, from looking at `mrt` we can see that this is pretty slow to run--and this was using a GPU! This performance time underscores the trade-off in using large language models at retrieval time: they may perform better, but could be much slower.

# Deep learning at indexing time: doc2query

Instead of using our large language models to rerank, another option is to use them at _indexing time_ to augment our documents. In class, we discussed one such option, doc2query, that augments an inverted index structure by predicting queries that may be used to search for the document, and appending those to the document text.

We can use doc2query using the `pyterrier_doc2query` package, which was loaded at the top.

### Loading a pre-trained model

We'll start by using a version of the doc2query model released by the authors that is trained on the MS MARCO collection.

In [17]:
if not os.path.exists("t5-base.zip"):
    !wget https://git.uwaterloo.ca/jimmylin/doc2query-data/raw/master/T5-passage/t5-base.zip
    !unzip t5-base.zip

We can load the model weights by specifying the checkpoint.

In [18]:
# doc2query = pyterrier_doc2query.Doc2Query('model.ckpt-1004000', batch_size=8)
doc2query = pyterrier_doc2query.Doc2Query('t5-base', batch_size=8)


/home/yianchu/.local/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5.py:164: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


### Running doc2queries on sample text

Let's see what queries it predicts for the sample document that we've made up:

In [19]:
df = pd.DataFrame([{"docno" : "d1", "text" :"The University of Michigan School of Information (UMSI) delivers innovative, elegant and ethical solutions connecting people, information and technology. The school was one of the first iSchools in the nation and is the premier institution studying and using technology to improve human computer interactions."}])
df.iloc[0].text

'The University of Michigan School of Information (UMSI) delivers innovative, elegant and ethical solutions connecting people, information and technology. The school was one of the first iSchools in the nation and is the premier institution studying and using technology to improve human computer interactions.'

In [20]:
doc2query_df = doc2query(df)
doc2query_df.iloc[0].querygen

'Michigan School of Information is the premier iSchool focusing on and using technology to improve human computer interaction.\nis an innovative and innovative college of Information (UMSI) is an elite school of information at the University of Michigan. The school is leading the nation in iSchools and human computer interaction.\nis the leading institution studying and using technology to improve people, information and technology. Michigan School of Information (UMSI) is the premiere institution researching and using technology to improve human computer interaction.'

Not too bad, though the questions are somewhat generic

### Loading an index of doc2query documents

Let's see how doc2query does on improving the performance in the TREC COVID data. Since indexing with doc2query takes a while (due to needing to run the deep learning models), we've provide an index with the text already added.

If you would like to index the collection with doc2query yourself (or use doc2query for your course project), you can use the following code:

```python
dataset = pt.get_dataset("irds:cord19/trec-covid")
indexer = (
  pyterrier_doc2query.Doc2Query('model.ckpt-1004000', doc_attr='abstract', batch_size=8, append=True) # aply doc2query on abstracts and append
  >> pt.apply.generic(lambda df: df.rename(columns={'abstract': 'text'}) # rename "abstract" column to "text" for indexing
  >> pt.IterDictIndexer("./doc2query_index_path")) # index the expanded documents
indexref = indexer.index(dataset.get_corpus_iter())
```


In [21]:
if not os.path.exists('doc2query_marco_cord19.zip'):
  !wget http://www.dcs.gla.ac.uk/~craigm/ecir2021-tutorial/doc2query_marco_cord19.zip
  !unzip doc2query_marco_cord19.zip
# doc2query_indexref = pt.IndexRef.of('./doc2query_index_path/data.properties')
doc2query_indexref = pt.IndexRef.of('./doc2query_index_path/data.properties')

--2022-11-01 03:00:41--  http://www.dcs.gla.ac.uk/~craigm/ecir2021-tutorial/doc2query_marco_cord19.zip
Resolving proxy.arc-ts.umich.edu (proxy.arc-ts.umich.edu)... 141.213.136.200
Connecting to proxy.arc-ts.umich.edu (proxy.arc-ts.umich.edu)|141.213.136.200|:3128... connected.
Proxy request sent, awaiting response... 302 Found
Location: https://www.dcs.gla.ac.uk/~craigm/ecir2021-tutorial/doc2query_marco_cord19.zip [following]
--2022-11-01 03:00:41--  https://www.dcs.gla.ac.uk/~craigm/ecir2021-tutorial/doc2query_marco_cord19.zip
Connecting to proxy.arc-ts.umich.edu (proxy.arc-ts.umich.edu)|141.213.136.200|:3128... connected.
ERROR: The certificate of ‘www.dcs.gla.ac.uk’ is not trusted.
ERROR: The certificate of ‘www.dcs.gla.ac.uk’ hasn't got a known issuer.
unzip:  cannot find or open doc2query_marco_cord19.zip, doc2query_marco_cord19.zip.zip or doc2query_marco_cord19.zip.ZIP.


Let's look at the results on TREC COVID by first merging the scores with the rankings

In [22]:
dataset = pt.get_dataset('irds:cord19/trec-covid')
pipeline = pt.BatchRetrieve(doc2query_indexref) % 1 >> pt.text.get_text(dataset, 'title')
res = pipeline(dataset.get_topics('title'))
res.merge(dataset.get_qrels(), how='left').head()

/home/yianchu/.local/lib/python3.9/site-packages/pyterrier/datasets.py:435: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(df.columns.difference(['qid','query']), 1, inplace=True)


,qid,docid,docno,rank,score,query,title,label,iteration
0,1,101299,jwmrgy5d,0,8.427298,coronavirus origin,COVID-19 in the heart and the lungs: could we ...,0.0,5
1,2,182167,g8grcy5j,0,13.922648,coronavirus response to weather changes,The Stirling Protocol – Putting the environmen...,0.0,4
2,3,85678,tl30wlpy,0,7.224180,coronavirus immunity,Receptor-dependent coronavirus infection of de...,NaN,NaN
3,4,145871,l5fxswfz,0,12.773362,how do people die from the coronavirus,Analysis on 54 Mortality Cases of Coronavirus ...,2.0,1.5
4,5,180990,3sepefqa,0,12.995980,animal models of covid 19,Current global vaccine and drug efforts agains...,0.0,4


What kind of queries does doc2query generate for the CORD19 documents?

In [23]:
df = pd.DataFrame(doc for doc in dataset.get_corpus_iter() if doc['docno'] in ('3sepefqa', 'l5fxswfz'))
df = df.rename(columns={'abstract': 'text'})
doc2query_df = doc2query(df)
for querygen, docno, text in zip(doc2query_df['querygen'], doc2query_df['docno'], df['text']):
    print(docno)
    print(querygen)
    print(text)

cord19/trec-covid documents:   0%|          | 0/192509 s<?, ?it/s]

l5fxswfz
Since the identification of the first case of coronavirus disease 2019 (COVID-19), the global number of confirmed cases as of March 15, 2020, is 156,400, with total deaths in 5,833 (3.7%) worldwide. Here, we summarize the morality data from February 19 when
Since the identification of the first case of COVID-19 in 2019, with the global number of confirmed cases as of March 15, 2020, is 156,400, with total death in 5,833 (3,7%). Here we summarize the morality data from February 19 when we identified the first case of CO
156,400 worldwide, and total death in 5,833 (3.7%) worldwide. Since the identification of the first case of coronavirus disease 2019 (COVID-19), the global number of confirmed cases as of March 15, 2020, is 156,400, with total death in 5,83
Since the identification of the first case of coronavirus disease 2019 (COVID-19), the global number of confirmed cases as of March 15, 2020, is 156,400, with total death in 5,833 (3.7%) worldwide. Here, we summarize the mora

## Evaluating the effects of doc2query

Here, we'll change our evaluation setup a bit from what we did before. Rather than compare two models for the same index, we'll instead compare the same model (BM25) with two different ways of indexing (two indices)! Our baseline will be an index of CORD19 without the doc2query additions.

Let's load a copy of the CORD19 index that we used earlier.

In [24]:
indexref = pt.IndexRef.of('./terrier_cord19/data.properties')

### Task 2: Write the `Experiment` to compare indices (5 points)
Run an `Experiment` using a `BM25` ranker that compares the indices `indexref` and `doc2query_indexref`.  Note that our doc2query model was trained on MS MARCO, which isn't the same kind of collection as CORD19, so this performance tells us how well that model can transfer to a new setting.

You should evaluate using the metrics "map", "ndcg", "ndcg_cut.10"

In [25]:
pt.Experiment(
  [pt.BatchRetrieve(indexref, wmodel="BM25"), pt.BatchRetrieve(doc2query_indexref, wmodel="BM25")],
  topics,
  qrels,
  eval_metrics=["map", "ndcg", "ndcg_cut.10"],
  names=["indexref", "doc2query_indexref"]
)
# TODO

,name,map,ndcg,ndcg_cut.10
0,indexref,0.195498,0.411103,0.644463
1,doc2query_indexref,0.196293,0.412673,0.623652


# Task 3: Train a new model! (30 points)

All of the prior exercises have had you working with either off-the-shelf models (not trained for IR) or models that someone else has trained for you. To give you a sense of how to train a model, your primary task in this notebook is to train a simple `knrm` model, which should be relatively efficient to train on a GPU. 

To keep thinsg simple, we'll use the same setup for CORD19 that we did in Part 2 (30 queries in train, 5 in dev, 15 queries in test) which is still relatively small for training a deep learning model but will get you started on the process. 

Your tasks are the following:
- Load the CORD19 dataset and split it into train, dev, and test. Your test set should have 15 queries, dev set 5 queries, and the rest in train. use a seed of 42 for the `random_state` to ensure consistent results with what we expected.
- Create a new `knrm` ranker and a pipeline that uses it
- Run an `Experiment` comparing four models:
  - a default `BatchRetrieve`, filtering to the top 100 results
  - BM25, filtering to the top 100 results
  - a pipeline that feeds the top 100 results of the default `BatchRetrieve` to your `knrm` model
  - a pipeline that feeds the top 100 results of BM25 to your `knrm` model
  
Your `Experiment` should evaluate with metrics `"map", "ndcg", 'ndcg_cut.10', 'P.10', 'mrt'`
  
We expect to see the `Experiment`'s results in the final cell. You are, of course, welcome to try training any of the fancier models to see how they do as well!

In [26]:
RANK_CUTOFF = 10
SEED=42

from sklearn.model_selection import train_test_split

dataset = pt.datasets.get_dataset('irds:cord19/trec-covid')
topics = dataset.get_topics(variant='description')
qrels = dataset.get_qrels()

tr_va_topics, test_topics = train_test_split(topics, test_size=15, random_state=SEED)
train_topics, valid_topics =  train_test_split(tr_va_topics, test_size=5, random_state=SEED)

#del knrm # clear out memory from KNRM (useful for GPU)
knrm = onir_pt.reranker('knrm', 'wordvec_hash', text_field='abstract')
bm25 = pt.BatchRetrieve(index, wmodel="BM25")


knrm_pipeline = (pt.BatchRetrieve(indexref) % 100 # get top 100 results
            >> pt.text.get_text(dataset, 'abstract') # fetch the document text
            >> pt.apply.generic(lambda df: df.rename(columns={'text': 'abstract'})) # rename columns
            >> knrm) # apply neural re-ranker
knrm_pipeline.fit(train_topics,qrels, valid_topics,qrels)


bm25_pipeline = (pt.BatchRetrieve(indexref, wmodel="BM25") % 100 # get top 100 results
            >> pt.text.get_text(dataset, 'abstract') # fetch the document text
            >> pt.apply.generic(lambda df: df.rename(columns={'text': 'abstract'})) # rename columns
            >> knrm) # apply neural re-ranker
bm25_pipeline.fit(train_topics,qrels, valid_topics,qrels)


pt.Experiment(
  [br % 100, bm25 % 100, knrm_pipeline, bm25_pipeline],
  test_topics,
  qrels,
  eval_metrics=["map", "ndcg", 'ndcg_cut.10', 'P.10', 'mrt'],
  names=["br", "bm25", "br >> knrm", "bm25 >> knrm"]
)
# TODO

/home/yianchu/.local/lib/python3.9/site-packages/pyterrier/datasets.py:435: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(df.columns.difference(['qid','query']), 1, inplace=True)


[2022-11-01 03:01:00,164][WordvecHashVocab][DEBUG] [starting] reading cached at /home/yianchu/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p
[2022-11-01 03:01:01,502][WordvecHashVocab][DEBUG] [finished] reading cached at /home/yianchu/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p [1.34s]
[2022-11-01 03:01:07,982][onir_pt][DEBUG] [starting] validation
[2022-11-01 03:01:07,984][onir_pt][ERROR] gpu=True, but CUDA is not available. Falling back on CPU.
[2022-11-01 03:01:07,986][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/125 s<?, ?it/s]

[2022-11-01 03:01:08,384][onir_pt][DEBUG] [finished] batches: s] [125it] [314.68it/s]
[2022-11-01 03:01:08,494][onir_pt][DEBUG] [finished] validation s]
[2022-11-01 03:01:08,496][onir_pt][INFO] pre-validation: 0.0073
[2022-11-01 03:01:08,533][onir_pt][ERROR] gpu=True, but CUDA is not available. Falling back on CPU.
[2022-11-01 03:01:08,537][onir_pt][DEBUG] [starting] training
[2022-11-01 03:01:08,541][onir_pt][DEBUG] [starting] train pairs


train pairs:   0%|          | 0/1024 s<?, ?it/s]

...

batches:   0%|          | 0/375 s<?, ?it/s]

[2022-11-01 03:08:07,743][onir_pt][DEBUG] [finished] batches: [1.22s] [375it] [306.12it/s]
[2022-11-01 03:08:08,658][onir_pt][ERROR] gpu=True, but CUDA is not available. Falling back on CPU.
[2022-11-01 03:08:08,660][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/375 s<?, ?it/s]

[2022-11-01 03:08:09,837][onir_pt][DEBUG] [finished] batches: [1.18s] [375it] [318.94it/s]


,name,map,ndcg,ndcg_cut.10,P.10,mrt
0,br,0.067211,0.165279,0.570299,0.633333,81.068571
1,bm25,0.073811,0.172321,0.594773,0.660000,60.066404
2,br >> knrm,0.065974,0.164825,0.540699,0.633333,144.810106
3,bm25 >> knrm,0.071050,0.170254,0.523204,0.620000,136.906838
